In [105]:
import networkx as nx
import numpy as np
from pgmpy.models.BayesianModel import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.factors.discrete import State
import random as random

In [125]:
student = BayesianModel([('diff', 'grade'), ('intel', 'grade')])
cpd_d = TabularCPD('diff', 2, [[0.6], [0.4]])
cpd_i = TabularCPD('intel', 2, [[0.7], [0.3]])
cpd_g = TabularCPD('grade', 3, [[0.3, 0.05, 0.9, 0.5], 
                                [0.4, 0.25,0.08, 0.3], 
                                [0.3, 0.7, 0.02, 0.2]],
               ['intel', 'diff'], [2, 2])
student.add_cpds(cpd_d, cpd_i, cpd_g)

In [183]:
def get_value_from_cpd(cpd_values,random_value):
    cpd_value_line = []
    current = 0;
    for i,cpd_value in enumerate(cpd_values):
        current += cpd_value
        if random_value<= current:
            return i

In [108]:
def get_value_from_cpd_evidence(cpd_values,evidence,random_value):
    k_length = len(cpd_values)
    cpd_value_line = []
    current = 0;
    for k in range(k_length):
        v = cpd_values[k][evidence[0]][evidence[1]]
        current += v
        if random_value<= current:
            return k
        #print(v)

In [109]:
model = student
sorted_order = list(nx.topological_sort(student))
sorted_order

['intel', 'diff', 'grade']

In [110]:
sample_size = 10

In [111]:
variable_count = len(student.nodes)
size = sample_size * variable_count
print(size)
r = [random.random() for _ in range(size)]
r

30


[0.21639469464692762,
 0.36772525663788824,
 0.9642977757032669,
 0.2623102433654687,
 0.8811447107621848,
 0.3027988127840141,
 0.8677340579313474,
 0.3010770339667802,
 0.9587862949949745,
 0.20804651851543432,
 0.06586264176661338,
 0.6001778443054191,
 0.4873802192831419,
 0.5706969430186097,
 0.0790388509015566,
 0.15921587383147606,
 0.2570482074553321,
 0.9490505553949541,
 0.02946730422650834,
 0.5436481200025295,
 0.12369462945831655,
 0.6791350831057534,
 0.9760082687964933,
 0.9939558217507388,
 0.43410021926032616,
 0.951157418007694,
 0.6115901002923657,
 0.8195585470458494,
 0.4580129739072849,
 0.8697658742149881]

In [130]:
k = 0
temp = dict()
for i in range(sample_size):
    for node in sorted_order:
            cpd = model.get_cpds(node)
            evidence = cpd.variables[:0:-1]
            #print(cpd.variable)
            #print('-----')
            #print(evidence)  
            v = None
            if evidence:
                #
                evidence_value = []
                for ev in evidence:
                    evidence_value.append(temp[ev])   
                v = get_value_from_cpd_evidence(cpd_g.values,evidence_value,r[k])
                
            else:
                #no evidence case
                v = get_value_from_cpd(cpd.values,r[k])
            
            temp[cpd.variables[0]] = v
            k+=1
    print(temp)

{'intel': 0, 'diff': 0, 'grade': 2}
{'intel': 0, 'diff': 1, 'grade': 0}
{'intel': 1, 'diff': 0, 'grade': 2}
{'intel': 0, 'diff': 0, 'grade': 1}
{'intel': 0, 'diff': 0, 'grade': 0}
{'intel': 0, 'diff': 0, 'grade': 2}
{'intel': 0, 'diff': 0, 'grade': 0}
{'intel': 0, 'diff': 1, 'grade': 2}
{'intel': 0, 'diff': 1, 'grade': 0}
{'intel': 1, 'diff': 0, 'grade': 2}


In [131]:
def prior_sampling(model,sample_size):
    
    sorted_order = list(nx.topological_sort(model))
    variable_count = len(model.nodes)
    size = sample_size * variable_count
    #print(size)
    r = [random.random() for _ in range(size)]
    k = 0
    temp = dict()
    sampled = []
    
    for i in range(sample_size):
        for node in sorted_order:
            cpd = model.get_cpds(node)
            evidence = cpd.variables[:0:-1]
            #print(evidence)  
            v = None
            if evidence:
                #
                evidence_value = []
                for ev in evidence:
                    evidence_value.append(temp[ev])   
                #v = get_value_from_cpd_evidence(cpd_g.values,evidence_value,r[k])
                v = get_value_from_cpd_evidence(cpd.values,evidence_value,r[k])
            else:
                #no evidence case
                v = get_value_from_cpd(cpd.values,r[k])
            
            temp[cpd.variables[0]] = v
            k+=1
            
        sampled.append(temp.copy())
        print(temp)
    return sampled

In [132]:
prior_sampling(student,sample_size=10)

{'intel': 1, 'diff': 0, 'grade': 1}
{'intel': 1, 'diff': 0, 'grade': 2}
{'intel': 1, 'diff': 0, 'grade': 2}
{'intel': 0, 'diff': 0, 'grade': 0}
{'intel': 0, 'diff': 0, 'grade': 2}
{'intel': 0, 'diff': 0, 'grade': 1}
{'intel': 0, 'diff': 0, 'grade': 1}
{'intel': 0, 'diff': 1, 'grade': 0}
{'intel': 1, 'diff': 1, 'grade': 1}
{'intel': 0, 'diff': 0, 'grade': 1}


[{'intel': 1, 'diff': 0, 'grade': 1},
 {'intel': 1, 'diff': 0, 'grade': 2},
 {'intel': 1, 'diff': 0, 'grade': 2},
 {'intel': 0, 'diff': 0, 'grade': 0},
 {'intel': 0, 'diff': 0, 'grade': 2},
 {'intel': 0, 'diff': 0, 'grade': 1},
 {'intel': 0, 'diff': 0, 'grade': 1},
 {'intel': 0, 'diff': 1, 'grade': 0},
 {'intel': 1, 'diff': 1, 'grade': 1},
 {'intel': 0, 'diff': 0, 'grade': 1}]

In [133]:
sample

[{'intel': 1, 'diff': 0, 'grade': 2},
 {'intel': 0, 'diff': 0, 'grade': 2},
 {'intel': 0, 'diff': 1, 'grade': 0},
 {'intel': 1, 'diff': 0, 'grade': 1},
 {'intel': 0, 'diff': 1, 'grade': 1},
 {'intel': 1, 'diff': 1, 'grade': 0},
 {'intel': 0, 'diff': 0, 'grade': 1},
 {'intel': 1, 'diff': 0, 'grade': 1},
 {'intel': 0, 'diff': 0, 'grade': 2},
 {'intel': 0, 'diff': 0, 'grade': 1}]

### Likelihood Weighted

In [180]:
k = 0
temp = dict()
for i in range(sample_size):
    for node in sorted_order:
            cpd = model.get_cpds(node)
            evidence = cpd.variables[:0:-1]
            #print(cpd.variable)
            #print('-----')
            #print(evidence)  

            
            v = None
            if evidence:
                #
                evidence_value = []
                for ev in evidence:
                    evidence_value.append(temp[ev])   
                v = get_value_from_cpd_evidence(cpd_g.values,evidence_value,r[k])
                
            else:
                #no evidence case
                v = get_value_from_cpd(cpd.values,r[k])
            
            temp[cpd.variables[0]] = v
            k+=1
    print(temp)

{'intel': 0, 'diff': 0, 'grade': 2}
{'intel': 0, 'diff': 1, 'grade': 0}
{'intel': 1, 'diff': 0, 'grade': 2}
{'intel': 0, 'diff': 0, 'grade': 1}
{'intel': 0, 'diff': 0, 'grade': 0}
{'intel': 0, 'diff': 0, 'grade': 2}
{'intel': 0, 'diff': 0, 'grade': 0}
{'intel': 0, 'diff': 1, 'grade': 2}
{'intel': 0, 'diff': 1, 'grade': 0}
{'intel': 1, 'diff': 0, 'grade': 2}


In [215]:
def get_weighted_from_cpd(cpd_values,random_value):
    cpd_value_line = []
    current = 0;
    for i,cpd_value in enumerate(cpd_values):
        current += cpd_value
        #print('cpdv',cpd_value)
        if random_value<= current:
            return cpd_value

In [237]:
def likelihood_weighted_sampling(model,given_evidence,sample_size):
    
    sorted_order = list(nx.topological_sort(model))
    variable_count = len(model.nodes)
    size = sample_size * variable_count
    #print(size)
    r = [random.random() for _ in range(size)]
    k = 0
    temp = dict()
    sampled = []
    
    for i in range(sample_size):
        w_list = []
        for node in sorted_order:
            cpd = model.get_cpds(node)
            evidence = cpd.variables[:0:-1]
            #print(evidence)  
            
            
            v = None
            w = 1.0
            
            if cpd.variable in given_evidences:
                v = given_evidences[cpd.variable]
                w_list.append(get_weighted_from_cpd(cpd.values,r[k]))
                #print('w',w_list)
            else:
                if evidence:
                #
                    evidence_value = []
                    for ev in evidence:
                        evidence_value.append(temp[ev])   
                        #print(evidence_value)
                        #v = get_value_from_cpd_evidence(cpd_g.values,evidence_value,r[k])
                    v = get_value_from_cpd_evidence(cpd.values,evidence_value,r[k])
                else:
                #no evidence case
                    v = get_value_from_cpd(cpd.values,r[k])
            
            
            temp[cpd.variables[0]] = v
            
            
            
            '''
            for wi in w_list:
                w *= wi
                print(w)
            temp['weighted'] = w
            '''
            
            k+=1
            
        
        print('w',w_list)
        w = 1.0
        for wi in w_list:
            w *= wi
            print('wi',wi)
        temp['weighted'] = w
            
        sampled.append(temp.copy())
        
        #print(temp)
    return sampled

In [139]:
given_evidence = [['intel','0']]
print(type(given_evidence))
likelihood_weighted_sampling([['intel','0']])

<class 'list'>


In [209]:
given_evidences = dict()
given_evidences['intel'] = 0
given_evidences['diff'] = 0
print(type(given_evidences))

variable = 'diff'

if variable in given_evidences:
    print('yes')
    print(given_evidences[variable])
else:
    print('no')


<class 'dict'>
yes
0


In [238]:
given_evidences = dict()
given_evidences['intel'] = 1
given_evidences['diff'] = 0
likelihood_weighted_sampling(student,given_evidences,20)


w [0.3, 0.4]
wi 0.3
wi 0.4
w [0.7, 0.6]
wi 0.7
wi 0.6
w [0.3, 0.6]
wi 0.3
wi 0.6
w [0.7, 0.6]
wi 0.7
wi 0.6
w [0.7, 0.6]
wi 0.7
wi 0.6
w [0.7, 0.6]
wi 0.7
wi 0.6
w [0.7, 0.6]
wi 0.7
wi 0.6
w [0.7, 0.4]
wi 0.7
wi 0.4
w [0.7, 0.6]
wi 0.7
wi 0.6
w [0.3, 0.4]
wi 0.3
wi 0.4
w [0.7, 0.4]
wi 0.7
wi 0.4
w [0.7, 0.4]
wi 0.7
wi 0.4
w [0.3, 0.4]
wi 0.3
wi 0.4
w [0.3, 0.6]
wi 0.3
wi 0.6
w [0.7, 0.4]
wi 0.7
wi 0.4
w [0.7, 0.6]
wi 0.7
wi 0.6
w [0.7, 0.6]
wi 0.7
wi 0.6
w [0.7, 0.4]
wi 0.7
wi 0.4
w [0.7, 0.6]
wi 0.7
wi 0.6
w [0.7, 0.6]
wi 0.7
wi 0.6


[{'intel': 1, 'diff': 0, 'grade': 1, 'weighted': 0.12},
 {'intel': 1, 'diff': 0, 'grade': 1, 'weighted': 0.42},
 {'intel': 1, 'diff': 0, 'grade': 2, 'weighted': 0.18},
 {'intel': 1, 'diff': 0, 'grade': 2, 'weighted': 0.42},
 {'intel': 1, 'diff': 0, 'grade': 1, 'weighted': 0.42},
 {'intel': 1, 'diff': 0, 'grade': 1, 'weighted': 0.42},
 {'intel': 1, 'diff': 0, 'grade': 1, 'weighted': 0.42},
 {'intel': 1, 'diff': 0, 'grade': 2, 'weighted': 0.27999999999999997},
 {'intel': 1, 'diff': 0, 'grade': 2, 'weighted': 0.42},
 {'intel': 1, 'diff': 0, 'grade': 2, 'weighted': 0.12},
 {'intel': 1, 'diff': 0, 'grade': 2, 'weighted': 0.27999999999999997},
 {'intel': 1, 'diff': 0, 'grade': 2, 'weighted': 0.27999999999999997},
 {'intel': 1, 'diff': 0, 'grade': 2, 'weighted': 0.12},
 {'intel': 1, 'diff': 0, 'grade': 1, 'weighted': 0.18},
 {'intel': 1, 'diff': 0, 'grade': 2, 'weighted': 0.27999999999999997},
 {'intel': 1, 'diff': 0, 'grade': 2, 'weighted': 0.42},
 {'intel': 1, 'diff': 0, 'grade': 2, 'weight